In [2]:
import pandas as pd
import numpy as np

In [3]:
# extract 

ld_add = pd.Timestamp.today()

tblpo_2023 = pd.ExcelFile(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - Portfolio Project\Access data\tblPO-jan-dec-2023-export-11-08-2025.xlsx")
tblpo_2024 = pd.ExcelFile(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - Portfolio Project\Access data\tblPO-jan-dec-2024-export-11-08-2025.xlsx")
tblpo_2025 = pd.ExcelFile(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - Portfolio Project\Access data\tblPO-jan-dec-2025-export-11-08-2025.xlsx")

access_tblpo_2023 = pd.read_excel(tblpo_2023, "tblPO")
access_tblpo_2024 = pd.read_excel(tblpo_2024, "tblPO")
access_tblpo_2025 = pd.read_excel(tblpo_2025, "tblPO")

In [4]:
# load po ref

po_numbers_2023_ref = pd.read_csv(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - GitHub Classroom Repo\angel_pythonmodulecode3\angel_jpcode3\data\raw\fnd_gfm_50117894-req-summary-jan-dec-2023-AN1446-ICT-consolidated.csv")
po_numbers_2024_ref = pd.read_csv(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - GitHub Classroom Repo\angel_pythonmodulecode3\angel_jpcode3\data\raw\fnd_gfm_50117895-req-summary-jan-dec-2024-AN1446-ICT-consolidated.csv")
po_numbers_2025_ref = pd.read_csv(r"C:\Users\angel\OneDrive\Documents\Multiverse DS ADF 2\Module 3\Module 3 - GitHub Classroom Repo\angel_pythonmodulecode3\angel_jpcode3\data\raw\fnd_gfm_50117898-req-summary-jan-aug-2025-AN1446-ICT-consolidated.csv")
po_ref_23_25 = pd.concat([po_numbers_2023_ref, po_numbers_2024_ref, po_numbers_2025_ref], ignore_index=True)
po_ref_stored = po_ref_23_25["OrderNumber"].dropna().astype("Int64").astype("str").tolist()

In [5]:
access_tblpo_2023.insert(0, "row_num", access_tblpo_2023.index + 1)
access_tblpo_2023.insert(1, "batch_load", ld_add)

In [6]:
access_tblpo_2023.shape

(758, 11)

In [7]:
access_tblpo_2023.dtypes

row_num                    int64
batch_load        datetime64[us]
POID                       int64
PONumber                  object
ProjectID                  int64
RaisedBy                  object
Site                      object
Notes                     object
SSMA_TimeStamp           float64
DateCreated       datetime64[ns]
Alemba                    object
dtype: object

In [8]:
access_tblpo_2023["SSMA_TimeStamp"].isna().sum()

758

In [9]:
access_tblpo_2024["SSMA_TimeStamp"].isna().sum()

631

In [10]:
access_tblpo_2025["SSMA_TimeStamp"].isna().sum()

295

In [11]:
access_tblpo_2023.insert(1, "batch_load", ld_add)
access_tblpo_2024.insert(1, "batch_load", ld_add)
access_tblpo_2025.insert(1, "batch_load", ld_add)

ValueError: cannot insert batch_load, already exists

In [12]:
access_tblpo_23_25 = pd.concat([access_tblpo_2023, access_tblpo_2024, access_tblpo_2025], ignore_index=True)

In [13]:
access_tblpo_23_25.insert(1, "batch_load", ld_add)

ValueError: cannot insert batch_load, already exists

In [14]:
parsed_access_tblpo_23_25 = access_tblpo_23_25.copy()

In [15]:
parsed_access_tblpo_23_25.dtypes

row_num                  float64
batch_load        datetime64[us]
POID                       int64
PONumber                  object
ProjectID                  int64
RaisedBy                  object
Site                      object
Notes                     object
SSMA_TimeStamp           float64
DateCreated       datetime64[ns]
Alemba                    object
dtype: object

In [16]:
parsed_access_tblpo_23_25["PONumber"] = parsed_access_tblpo_23_25["PONumber"].str.strip().replace("", np.nan)
parsed_access_tblpo_23_25["RaisedBy"] = parsed_access_tblpo_23_25["RaisedBy"].str.strip().replace("", np.nan)
parsed_access_tblpo_23_25["Site"] = parsed_access_tblpo_23_25["Site"].str.strip().replace("", np.nan)
parsed_access_tblpo_23_25["Notes"] = parsed_access_tblpo_23_25["Notes"].str.strip().replace("", np.nan)
parsed_access_tblpo_23_25["Alemba"] = parsed_access_tblpo_23_25["Alemba"].str.strip().replace("", np.nan)
parsed_access_tblpo_23_25["parsed_POID"] = pd.to_numeric(parsed_access_tblpo_23_25["POID"], errors="coerce").astype("Int64")
parsed_access_tblpo_23_25["parsed_ProjectID"] = pd.to_numeric(parsed_access_tblpo_23_25["ProjectID"], errors="coerce").astype("Int64")

In [17]:
parsed_access_tblpo_23_25.columns

Index(['row_num', 'batch_load', 'POID', 'PONumber', 'ProjectID', 'RaisedBy',
       'Site', 'Notes', 'SSMA_TimeStamp', 'DateCreated', 'Alemba',
       'parsed_POID', 'parsed_ProjectID'],
      dtype='object')

In [18]:
import datetime

mandatory_fields = ["parsed_POID", "PONumber", "parsed_ProjectID", "RaisedBy", "Site", "Notes", "DateCreated", "Alemba"]
source_system = "ACCESS"
system_table = "tblPO"
data_period = "23-24 to 24-25"
date_checked = datetime.date.today()



allowed_start_date = pd.to_datetime("2023-01-01")
allowed_end_date = pd.to_datetime("2025-08-11")
allowed_sites = ["RLH", "WCH", "SBH", "MEH", "NUH", "WXH"]

In [23]:
# tblPO errors

summary_records = []
for field in mandatory_fields:
    missing_count = parsed_access_tblpo_23_25[field].isna().sum()
    summary_records.append({
        "error_type": "Missing Field",
        "source_system": source_system,
        "system_table": system_table,
        "data_period": data_period,
        "date_checked": date_checked,
        "field_name": field,
        "error_count": missing_count
    })




# format error / invalid format #revising tbc
for field in mandatory_fields:
    if not field.startswith("parsed_"):
        continue
    raw_field = field.replace("parsed_", "")
    if field not in parsed_access_tblpo_23_25.columns or raw_field not in parsed_access_tblpo_23_25:
        continue
    format_error_count = parsed_access_tblpo_23_25[parsed_access_tblpo_23_25[raw_field].notna() & parsed_access_tblpo_23_25[field].isna()].shape[0]
    summary_records.append({
        "error_type": "Invalid Format",
        "source_system": source_system,
        "system_table": system_table,
        "data_period": data_period,
        "date_checked": date_checked,
        "field_name": field,
        "error_count": format_error_count
    })



# out of range error - site
out_of_range_site_count = parsed_access_tblpo_23_25[
    parsed_access_tblpo_23_25["Site"].notna() &
    ~parsed_access_tblpo_23_25["Site"].isin(allowed_sites)].shape[0]

summary_records.append({
    "error_type": "Out of Range Values",
    "source_system": source_system,
    "system_table": system_table,
    "data_period": data_period,
    "date_checked": date_checked,
    "field_name": "Site",
    "error_count": out_of_range_site_count
})



# out of range error - date created / date entry

out_of_range_date_count = parsed_access_tblpo_23_25[parsed_access_tblpo_23_25["DateCreated"].notna() &
(
    (parsed_access_tblpo_23_25["DateCreated"] < allowed_start_date) |
    (parsed_access_tblpo_23_25["DateCreated"] > allowed_end_date)
)].shape[0]

summary_records.append({
    "error_type": "Out of Range Dates",
    "source_system": source_system,
    "system_table": system_table,
    "data_period": data_period,
    "date_checked": date_checked,
    "field_name": "DateCreated",
    "error_count": out_of_range_date_count
})



# Duplicate PO numbers - flag this as well just in case there's need to revise or amend in data entry process

df_a = parsed_access_tblpo_23_25
po = "PONumber"
duplicate_count_po = df_a[po].dropna().duplicated().sum()

if duplicate_count_po > 0:
    summary_records.append({
    "error_type": "Duplicate PO Number",
    "source_system": source_system,
    "system_table": system_table,
    "data_period": data_period,
    "date_checked": date_checked,
    "field_name": "PONumber",
    "error_count": duplicate_count_po
    })


# Unexpected Value - PO doesnt exist in oracle data list fo PO orders
# set format rule or refer to PO numbers oracle? TBC



maskfilter_ordernum = ~parsed_access_tblpo_23_25["PONumber"].isin(po_ref_stored)

unexpected_ordernum_count = parsed_access_tblpo_23_25.loc[
    maskfilter_ordernum & parsed_access_tblpo_23_25["PONumber"].notna()].shape[0]

if unexpected_ordernum_count > 0:
    summary_records.append({
    "error_type": "Unexpected PO Number",
    "source_system": source_system,
    "system_table": system_table,
    "data_period": data_period,
    "date_checked": date_checked,
    "field_name": "PONumber",
    "error_count": unexpected_ordernum_count
    })

In [24]:
parsed_access_tblpo_23_25['PONumber'].duplicated().sum()

219

In [25]:
parsed_access_tblpo_23_25['PONumber'].dropna().duplicated().sum()

189

In [26]:
format_error_count

0

In [27]:
unexpected_ordernum_count

349

In [32]:
error_summary_access_tblpo = pd.DataFrame(summary_records)





error_summary_access_tblpo["field_name"] = error_summary_access_tblpo["field_name"].str.replace(r"^parsed_", "", regex=True)



error_summary_access_tblpo = error_summary_access_tblpo[error_summary_access_tblpo["error_count"] > 0]

In [33]:
error_summary_access_tblpo

,error_type,source_system,system_table,data_period,date_checked,field_name,error_count
1,Missing Field,ACCESS,tblPO,23-24 to 24-25,2025-09-21,PONumber,31
3,Missing Field,ACCESS,tblPO,23-24 to 24-25,2025-09-21,RaisedBy,39
4,Missing Field,ACCESS,tblPO,23-24 to 24-25,2025-09-21,Site,516
5,Missing Field,ACCESS,tblPO,23-24 to 24-25,2025-09-21,Notes,764
7,Missing Field,ACCESS,tblPO,23-24 to 24-25,2025-09-21,Alemba,477
10,Out of Range Values,ACCESS,tblPO,23-24 to 24-25,2025-09-21,Site,85
12,Duplicate PO Number,ACCESS,tblPO,23-24 to 24-25,2025-09-21,PONumber,189
13,Unexpected PO Number,ACCESS,tblPO,23-24 to 24-25,2025-09-21,PONumber,349


In [34]:
error_summary_access_tblpo.to_csv("tblpo_access_error_report.csv", index=False)